In [75]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [76]:
url = f'https://graph.instagram.com/me/media?fields=id,caption,media_type,media_url,timestamp&access_token=IGQWRONmdaelNlOTB3cjFzcTFTd25tLUloYWRySHg4SlhicmlFSW5OV1pWcnNYU2YtYkNkSFJWTThwcnpNTW02d19ZATzVrMUJrMVFpcXpFYm5JeDFfODFDaDdfQkRDMHJuOXpXd0c2VE1Ta3JBb1lHalpUUG5PTDgZD'

In [77]:
access_token='IGQWRONmdaelNlOTB3cjFzcTFTd25tLUloYWRySHg4SlhicmlFSW5OV1pWcnNYU2YtYkNkSFJWTThwcnpNTW02d19ZATzVrMUJrMVFpcXpFYm5JeDFfODFDaDdfQkRDMHJuOXpXd0c2VE1Ta3JBb1lHalpUUG5PTDgZD'

In [78]:
response = requests.get(url)

In [79]:
if response.status_code == 200:
    data = response.json()
else:
    print("Error making the API request:", response.text)
    data = None

In [80]:
data

{'data': [{'id': '18227088265280124',
   'caption': "I wish I could see a way out, but the path forward is so unclear. I know I have to keep moving, but some days it's just too hard. The silence can be deafening, and I often find myself wishing for a sign, something to show me that I'm not alone in this. I keep hoping that tomorrow will be better, but each new day feels like a rerun of the last. I just want to feel something again.",
   'media_type': 'IMAGE',
   'media_url': 'https://scontent.cdninstagram.com/v/t51.29350-15/441248127_1158973988624621_3187270308235607167_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=18de74&_nc_ohc=1heq8_CtWDQQ7kNvgGWp9Mt&_nc_oc=AdheK3FuHPjAwRXrUHkkKz8AhnVBVquy_pZ4n4GBQLEd7-L_H9Bsyzsl-_KltSDgcv6tNpoAWTqTlfEupuveNag6&_nc_ht=scontent.cdninstagram.com&edm=ANo9K5cEAAAA&oh=00_AfCOmQXcq0UoUN86hyQX4KMvasXrCAcVKRYOY21lWvx-ag&oe=664246A8',
   'timestamp': '2024-05-04T13:05:43+0000'},
  {'id': '18001978385605323',
   'caption': "There’s an emptiness that follows when a relati

In [81]:
if data:
    df = pd.DataFrame(data['data'])  # Assuming 'data' is a dictionary containing 'data' key with a list of records
else:
    df = pd.DataFrame(columns=['id', 'caption','mediatype','media_url','timestamp'])


In [82]:
print(df)

                  id                                            caption  \
0  18227088265280124  I wish I could see a way out, but the path for...   
1  18001978385605323  There’s an emptiness that follows when a relat...   
2  18028155728062543  Life often feels like an uphill climb, with no...   
3  18063719863538580  The bottle seemed like a quick escape from my ...   
4  18008219390203733  What started as a game turned into something d...   
5  18014434958096049  When you find out she thought you were playing...   
6  18019842629009943  Dreams are like fragile threads, easily broken...   
7  18049687408639342                           It gets tougher everyday   

  media_type                                          media_url  \
0      IMAGE  https://scontent.cdninstagram.com/v/t51.29350-...   
1      IMAGE  https://scontent.cdninstagram.com/v/t51.29350-...   
2      IMAGE  https://scontent.cdninstagram.com/v/t51.29350-...   
3      IMAGE  https://scontent.cdninstagram.com/v/t51.29

In [83]:
df.to_csv('instagram_data.csv', index=False)

In [84]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Model

from keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, Concatenate, Dot, Activation

In [85]:
from ocr import get_text

In [86]:
training_data = pd.read_csv('training_data.csv')

In [87]:
instagram_data = pd.read_csv('instagram_data.csv')


In [88]:
def fetch_carousel_items(album_id, access_token):
    album_url = f'https://graph.instagram.com/media/{album_id}/children?fields=id,media_type,media_url,caption&access_token={access_token}'
    response = requests.get(album_url)

    if response.status_code != 200:
        print("Error fetching album data:", response.text)
        return None

    return response.json()

In [90]:
new_data = []
for index, row in df.iterrows():
    new_row = {'id': row['id'], 'timestamp': row['timestamp']}
    flag = 0
    if row['media_type'] == 'IMAGE':
        # Perform OCR on the image URL
        ocr_text = get_text(row['media_url'])
        if ocr_text:
            new_row['text'] = ocr_text
            flag = 1
        else:
        # Use caption as text
            new_row['text'] = row['caption']    
    else:
        # Use caption as text
        new_row['text'] = row['caption']
    new_data.append(new_row)
    if flag == 1:
        new_row_2 = {'id': row['id'], 'timestamp': row['timestamp'],'text':row['caption']}
        new_data.append(new_row_2)

# Create new DataFrame
new_df = pd.DataFrame(new_data)
print(new_df)

                   id                 timestamp  \
0   18227088265280124  2024-05-04T13:05:43+0000   
1   18227088265280124  2024-05-04T13:05:43+0000   
2   18001978385605323  2024-05-04T13:05:02+0000   
3   18001978385605323  2024-05-04T13:05:02+0000   
4   18028155728062543  2024-05-04T13:01:15+0000   
5   18028155728062543  2024-05-04T13:01:15+0000   
6   18063719863538580  2024-05-04T12:53:06+0000   
7   18063719863538580  2024-05-04T12:53:06+0000   
8   18008219390203733  2024-05-04T12:52:00+0000   
9   18008219390203733  2024-05-04T12:52:00+0000   
10  18014434958096049  2024-05-04T12:50:31+0000   
11  18014434958096049  2024-05-04T12:50:31+0000   
12  18019842629009943  2024-05-04T12:48:04+0000   
13  18019842629009943  2024-05-04T12:48:04+0000   
14  18049687408639342  2024-05-04T10:29:34+0000   
15  18049687408639342  2024-05-04T10:29:34+0000   

                                                 text  
0   66\n\nlam bent, but not broken. |\nam scarred,...  
1   I wish I could s

In [91]:
new_df.to_csv('test.csv')

In [92]:
new_df.dropna(subset=['text'], inplace=True)
X_train = training_data['caption'].astype(str)
y_train = training_data['depression_label']
X_test = new_df['text'].astype(str)

In [93]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [94]:
max_len = 100  # Define your maximum sequence length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [95]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = Dense(1, activation='tanh')(lstm_layer)
attention = Activation('sigmoid')(attention)
context = Dot(axes=1)([attention, lstm_layer])
output_layer = Dense(1, activation='sigmoid')(context)

model = Model(inputs=[input_layer], outputs=output_layer)

In [96]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [97]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=64)

Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.8048 - loss: 0.5943
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.9616 - loss: 0.1208
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.9830 - loss: 0.0629
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.9896 - loss: 0.0360
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - accuracy: 0.9905 - loss: 0.0328
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.9925 - loss: 0.0229
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.9918 - loss: 0.0326
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - accuracy: 0.9946 - loss: 0.0219
Epoch 9/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.9959 - loss: 0.0135
Epoch 10/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.9977 - loss: 0.0093


In [100]:
predicted_labels = model.predict(X_test_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [101]:
new_df['predicted_depression_label'] = predicted_labels.reshape(-1)


In [102]:
depressing_captions = new_df[new_df['predicted_depression_label'] > 0.9]['text']

depression_text_with_one = new_df[new_df['predicted_depression_label'] == 1]['text'].iloc[0] if not new_df[new_df['predicted_depression_label'] == 1].empty else None

if depression_text_with_one is not None:
    print("\nDepression text with label 1:")
    print(depression_text_with_one)
else:
    print("\nNo text with predicted depression label equal to 1 found.")



Depression text with label 1:
Everyday it gets harder.

Everyday it's like a nightmare
progressing on.

Everyday is another prayer
echohing from my bones
asking God
to take me now.

Everyday I close up more.

Everyday it gets more impossible.

Everyday I feel myself die some more.

Everyday | lay on my bed
wondering how
I'll leave this hell.

I've never felt so much pain.



In [103]:
for text in depressing_captions:
    print(text)


66

lam bent, but not broken. |
am scarred, but not
disfigured. | am sad, but not
hopeless. | am tired, but not
powerless. | am angry, but
not bitter. | am depressed,
but not giving up

A HopeQure

Life often feels like an uphill climb, with no clear end in sight. Every day brings a new set of challenges, and sometimes it feels like I'm stuck in a cycle that never seems to break. The weight of expectations, both my own and those of others, can be so crushing. It's hard to breathe when every step feels like it takes me further from where I want to be.
What started as a game turned into something darker. Gambling addiction turned my dreams into debts. Every bet took me further from the life I wanted
le everything changes aro
ne | stay forever the same
\ttempt now and then. Cu
renever the urge hits as tl
cle repeats. when will it e\

end 7)

Depression-cc yn.tumblir.com

Confession #5398

Everyday it gets harder.

Everyday it's like a nightmare
progressing on.

Everyday is another prayer


In [105]:
%pip install openai==0.28
import openai
import os
openai.api_key = 'insert_your_api_key'


def get_gpt_response(text):
    # Create a messages list for the Chat Completion API
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that provides compassionate responses and includes links to online mental health resources."
        },
        {
            "role": "user",
            "content": f"The following text expresses feelings of sadness or depression:\n{text}\n"
        }
    ]
    

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7,  
        max_tokens=100,  
        top_p=1.0,  
        frequency_penalty=0,  
        presence_penalty=0  
    )
    
   
    generated_response = response.choices[0].message['content']
    

    print("Generated Response:")
    print(generated_response)
    

    return generated_response


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [106]:
response = get_gpt_response(text)

Generated Response:
I'm sorry to hear that you're feeling this way. It sounds like you're going through a really tough time, but please know that you are not alone. It's important to reach out for help and support when you're feeling overwhelmed with sadness or depression.

If you're comfortable, talking to a mental health professional can be very beneficial. They can provide you with coping strategies, support, and guidance to help you through this difficult time. You can also consider reaching out to a trusted friend or


In [107]:
print(response)

I'm sorry to hear that you're feeling this way. It sounds like you're going through a really tough time, but please know that you are not alone. It's important to reach out for help and support when you're feeling overwhelmed with sadness or depression.

If you're comfortable, talking to a mental health professional can be very beneficial. They can provide you with coping strategies, support, and guidance to help you through this difficult time. You can also consider reaching out to a trusted friend or


In [108]:
print(predicted_labels)

[[[9.9998784e-01]]

 [[2.4398874e-01]]

 [[1.5543386e-01]]

 [[3.6070293e-01]]

 [[9.2759803e-02]]

 [[9.9999535e-01]]

 [[1.9927046e-01]]

 [[6.9403285e-01]]

 [[1.1790399e-01]]

 [[9.9932855e-01]]

 [[9.9996775e-01]]

 [[1.5434223e-05]]

 [[4.4897240e-01]]

 [[8.9659011e-03]]

 [[1.0000000e+00]]

 [[4.3256730e-01]]]


In [109]:
new_df

,id,timestamp,text,predicted_depression_label
0,18227088265280124,2024-05-04T13:05:43+0000,"66\n\nlam bent, but not broken. |\nam scarred,...",0.999988
1,18227088265280124,2024-05-04T13:05:43+0000,"I wish I could see a way out, but the path for...",0.243989
2,18001978385605323,2024-05-04T13:05:02+0000,(i\n\n“Relationships are like —\nglass. Someti...,0.155434
3,18001978385605323,2024-05-04T13:05:02+0000,There’s an emptiness that follows when a relat...,0.360703
4,18028155728062543,2024-05-04T13:01:15+0000,0. Xa Se St. AS MN\n®6\n\nTears are words the\...,0.092760
5,18028155728062543,2024-05-04T13:01:15+0000,"Life often feels like an uphill climb, with no...",0.999995
6,18063719863538580,2024-05-04T12:53:06+0000,It’s not always the\ntears that measure the\np...,0.199270
7,18063719863538580,2024-05-04T12:53:06+0000,The bottle seemed like a quick escape from my ...,0.694033
8,18008219390203733,2024-05-04T12:52:00+0000,"66\nKeeping a lot to\nmyself, because\nit is h...",0.117904
9,18008219390203733,2024-05-04T12:52:00+0000,What started as a game turned into something d...,0.999329


In [110]:
new_df.to_csv('forpurp.csv')

In [111]:
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
import numpy as np


training_data = pd.read_csv('training_data.csv')

X_train = training_data['caption'].astype(str)
y_train = training_data['depression_label']


X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  
cross_val_results = []

for train_index, val_index in kf.split(X_train_pad):
    X_train_fold, X_val_fold = X_train_pad[train_index], X_train_pad[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    model = Model(inputs=[input_layer], outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

    history = model.fit(
        X_train_fold,
        y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        epochs=10,
        batch_size=64,
        callbacks=[early_stopping],
    )

    cross_val_results.append((history.history['val_loss'][-1], history.history['val_accuracy'][-1]))

print("Cross-Validation Results:")
for i, result in enumerate(cross_val_results):
    print(f"Fold {i + 1}: Loss = {result[0]}, Accuracy = {result[1]}")


Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 11s 72ms/step - accuracy: 0.9949 - loss: 0.0211 - val_accuracy: 0.9910 - val_loss: 0.0327
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.9952 - loss: 0.0228 - val_accuracy: 0.9935 - val_loss: 0.0162
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 7s 70ms/step - accuracy: 0.9993 - loss: 0.0037 - val_accuracy: 0.9929 - val_loss: 0.0210
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.9994 - loss: 0.0035 - val_accuracy: 0.9948 - val_loss: 0.0279
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.9992 - loss: 0.0071 - val_accuracy: 0.9942 - val_loss: 0.0376
Epoch 5: early stopping
Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 12s 72ms/step - accuracy: 0.9970 - loss: 0.0241 - val_accuracy: 1.0000 - val_loss: 2.6554e-04
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.9972 - loss: 0.0141 - val_accuracy: 0.9994 - val_loss: 0.0070
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 64ms/step - accuracy: 0.9978 - loss: 0.0

In [ ]:

# Manual review
num_samples = 10  
sampled_df = new_df.sample(num_samples, random_state=42)

for index, row in sampled_df.iterrows():
    print(f"Caption: {row['text']}")
    print(f"Predicted Depression Label: {row['predicted_depression_label']}")


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred_train = (model.predict(X_train_pad) > 0.5).astype(int).reshape(-1)

accuracy = accuracy_score(y_train, y_pred_train)
precision = precision_score(y_train, y_pred_train)
recall = recall_score(y_train, y_pred_train)
f1 = f1_score(y_train, y_pred_train)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

cm = confusion_matrix(y_train, y_pred_train)
print("Confusion Matrix:")
print(cm)


In [ ]:
import random

# Introduce noise into the training data
noisy_data = X_train + ' ' + pd.Series([''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=10)) for _ in range(len(X_train))])

# Re-tokenize and re-pad with the noisy data
noisy_data_seq = tokenizer.texts_to_sequences(noisy_data)
noisy_data_pad = pad_sequences(noisy_data_seq, maxlen=max_len, padding='post')

# Predict with the noisy data
y_pred_noisy = (model.predict(noisy_data_pad) > 0.5).astype(int).reshape(-1)

# Calculate metrics for the noisy data
accuracy_noisy = accuracy_score(y_train, y_pred_noisy)
precision_noisy = precision_score(y_train, y_pred_noisy)
recall_noisy = recall_score(y_train, y_pred_noisy)
f1_noisy = f1_score(y_train, y_pred_noisy)

print("Metrics with Noisy Data:")
print(f"Accuracy: {accuracy_noisy:.2f}")
print(f"Precision: {precision_noisy:.2f}")
print(f"Recall: {recall_noisy:.2f}")
print(f"F1-Score: {f1_noisy:.2f}")


In [ ]:

def plot_attention(caption, attention_weights, tokenizer, max_len):
    tokens = tokenizer.texts_to_sequences([caption])[0]
    words = [tokenizer.index_word[tok] for tok in tokens]

    if len(words) < max_len:
        words += [""] * (max_len - len(words))

    plt.figure(figsize=(10, 6))
    plt.plot(range(max_len), attention_weights, label='Attention Weights')
    plt.xticks(ticks=range(max_len), labels=words, rotation=90)
    plt.title('Attention Weights for Caption')
    plt.xlabel('Word Position')
    plt.ylabel('Attention Weight')
    plt.show()

attention_weights_layer = model.layers[-3].output  
attention_model = Model(inputs=model.input, outputs=attention_weights_layer)

for idx, row in new_df.iterrows():
    caption = row['text']
    caption_seq = tokenizer.texts_to_sequences([caption])
    caption_pad = pad_sequences(caption_seq, maxlen=max_len, padding='post')
    attention_output = attention_model.predict(caption_pad)[0]
    plot_attention(caption, attention_output, tokenizer, max_len)
